<a href="https://colab.research.google.com/github/SRARNAB7/HDS_5230_07_Arnab/blob/main/Week%2010/stream_service_data_science_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
import fastapi
from typing import Optional

In [4]:


_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"]
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [5]:
def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
    testacc=accuracy_score(y_test, y_pred)
    testrecall=recall_score(y_test, y_pred)
    testprecision=precision_score(y_test, y_pred)
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [6]:
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [12]:
app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n:int):
    print ("RETRAINING over last {}".format(last_n))
    #print(_x)
    #print(_y)
    X = np.rec.fromarrays([_x, _y])[:(-1*last_n)]
    Y = np.rec.fromarrays([_result])[:(-1*last_n)]

    #print(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    print(X_train)
    print(y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(k=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

#@app.get("/evaluate_model/")
#def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

# NEW CODE ADDED FOR RETRAIN

In [2]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/predict/")
def predict(id: int = 0, x: float = 0, y: float = 0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([[x, y]]))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return {"prediction": prediction[0]}


@app.get("/result/")
def result(id: int, result: str):
    _result.insert(id, result)
    return {"message": f"Copied result for {id}"}


@app.get("/retrain/")
def retrain(last_n: int):
    global model

    # Check if enough data exists
    if len(_x) < last_n or len(_result) < last_n:
        return {
            "error": f"Not enough data to retrain. Required: {last_n}, Available: {len(_x)}"
        }

    print(f"RETRAINING over last {last_n} samples")
    X = np.array(list(zip(_x[-last_n:], _y[-last_n:])))
    Y = np.array(_result[-last_n:])

    if len(Y) < 2:
        return {"error": "Need at least 2 samples for training and testing."}

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {
        "message": "Model retrained",
        "accuracy": testacc,
        "recall": testrecall,
        "precision": testprecision
    }


@app.get("/new_model/")
def new_model(model_type: str, last_n: int, strategy: Optional[str] = None, k: Optional[int] = 3):
    global model

    print(model_type, last_n, strategy, k)

    if model_type == "Dummy":
        model = DummyClassifier(strategy=strategy)
    elif model_type == "KNN":
        model = KNeighborsClassifier(n_neighbors=k)

    retrain_result = retrain(last_n=last_n)

    return {
        "message": f"Model switched to {model_type}",
        "train_result": retrain_result
    }

In [3]:
#print(predict(1, 1, 1))
#result(1, True)
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 3


NameError: name '_x' is not defined

In [16]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [17]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())